# CMS Convexity Adjustment
------------------
> **Idriss Afra**

This project aims to compute the CMS Convexity Adjustment and price CMS Caplets / Floorlets.

## Model Description

CMS derivatives are very popular products nowadays because they provide a market efficient access to long dated interest rates. In fact, their payoffs depend on a swap rate of a constant (fixed) maturity $f\left(S(T_f, T_0, T_1)\right)$ where :    
* $f$ is the payoff function. Example : $f(s)=s$ for CMS forwards
* $S$ is the swap rate
* $T_f$ is the fixing date
* $T_0$ is the start date $\left(T_0 = T_f + 2BD\right)$
* $T_1$ is the end date such as $T_1 - T_0$ is the swap's tenor

Let us assume a cash-flow of $f\left(S(T_f, T_0, T_1)\right)$ paid at $T_p$, its $PV$ under the $T_p$-forward measure $Q^{T_p}$ is :

$$
PV = B(0, Tp) \times E^{Q^{T_p}} \left(f\left(S(T_f, T_0, T_1)\right)\right)
$$

Where : $B(0, Tp)$ is the zero-coupon price up to $T_p \in [T_0, T_1]$

We use the Radon-Nikodym method to switch to the level measure $Q^{LVL}$ under which the $S(t, T_0, T_1)$ is martingale :    

\begin{equation}
\begin{split}
PV & = B(0, Tp) \times E^{Q^{T_p}} \left( \frac{f\left(S(T_f, T_0, T_1)\right) \times B(T_f, T_p) }{B(T_f, T_p)} \right) \\
& = LVL(0,T_0, T_1) \times E^{Q^{LVL}} \left( \frac{f\left(S(T_f, T_0, T_1)\right) \times B(T_f, T_p) }{LVL(T_f,T_0, T_1)} \right) \\
\end{split}
\end{equation}




## TSR Model : Mean Reversion Linear

The terminal swap rate (TSR) approach assumes that the zero-coupon prices $\left(B(T_f, T^p_i)\right)_{T^p_i \in SwapPayDates}$ are linked functionally to the driving swap rate $S(T_f, T_0, T_1)$. Therefore, it exists a function $g$ under this assumption such as :    

$$
\frac{B(T_f, T_p)}{LVL(T_f,T_0, T_1)} ≈ g\left(S(T_f, T_0, T_1)\right)
$$

We use the Mean Reversion Linear TSR model in this project : $g(s) = a\left(T_p\right) \times s + b\left(T_p\right)$

In this case :
\begin{equation}
\begin{split}
a\left(T_p\right) & = \frac{d}{dS(T_f, T_0, T_1)} \left( \frac{B(T_f, T_p)}{LVL(T_f,T_0, T_1)}  \right) \\
& = \frac{d}{dS(T_f)} \left( \frac{B(T_f, T_p)}{∑_{T^p_i \in SwapPayDates} DCF_i × B(T_f, T^p_{i}) } \right) \\
\end{split}
\end{equation}

We can rewrite this derivative in the context of the HW-1F model as :    

$$
a\left(T_p\right) = \frac{d}{dX_{T_f}} \left( \frac{B(T_f, T_p, X_{T_f})}{∑_{T^p_i \in SwapPayDates} DCF_i × B(T_f, T^p_{i}, X_{T_f})}  \right)_{S(T_f, X_{T_f}) = S(0)} × \frac{1}{\left( \frac{d S(T_f, X_{T_f})}{dX_{T_f}}\right)_{S(T_f, X_{T_f}) = S(0)}}
$$

Where :    
* $X_t = r(t) - f(0, t) \text{}$ : $r_t$ is the short rate and $f(0, t) = -\frac{dLOG\left(B(0, t)\right)}{dt} $ is the instantaneous forward rate $\left(X_0 = 0\right)$
* $B(t, T, X_t) = \frac{B(0, T)}{B(0, t)} × EXP\left( -\frac{1}{2} × β(t, T)^{2} × Φ(t) - β(t, T) × X_t \right)$ : The HW-1F zero-coupon price
* $β(t, T) = \frac{1 - e^{-λ (T-t)}}{λ}$ : $λ$ is the HW-1F mean reversion
* $Φ(t) = ∫^{t}_{0} σ(s)^{2} e^{-2λ(t-s)} ds$ : $σ(t)$ is the short rate volatility

By using for all $T_0 < T ≤ T_1$ the following approximation :

$$
B(T_f, T, X_{T_f})_{S(T_f, X_{T_f}) = S(0)} ≈ \frac{B(0, T)}{B(0, T_f)}
$$

We get the following derivation result :    

$$
a\left(T_p\right) = \frac{B(0, T_p) × (γ - \beta(T_f, T_p))}{B(0, T_N)×β(T_f, T_N)+LVL(0, T_0, T_1)×S(0, T_0, T_1)×γ}
$$

Where :

$$
γ = \frac{∑_{T^p_i \in SwapPayDates} DCF_i × B(0, T^p_{i})×β(T_f, T^p_{i})}{LVL(0, T_0, T_1)}
$$

On the other hand, for $T_f=0$ :    

$$
b\left(T_p\right) = \frac{B(0, T_p)}{LVL(0,T_0, T_1)} - a × S(0, T_0, T_1)
$$

In [1]:
class tsr_model :
    """
    TSR model : Mean Reversion Linear flavor
    """
    def __init__(self, df, mean_reversion=0.015) :
        """
        Init method : df is the discount curve function and the HW-1F mean reversion is defaulted to 1.50%
        """
        self.df = df
        self.mean_reversion = mean_reversion

    def beta(self, t, T) :
        """
        HW-1F Beta function
        """
        return (1 - np.exp(-self.mean_reversion * (T - t))) / self.mean_reversion

    def tsr_coeffs(self, expiry, tenor, pay_date) :
        """
        TSR coefficients formulas
        """
        # Dates
        start_time = expiry + 2./365.
        # Fixed leg payment dates : Annually
        pay_times = np.arange(start_time + 1, expiry + tenor + 1)
        year_fractions = np.ones(len(pay_times))

        # Intermidiate Variables
        df_start_time = self.df(start_time)
        df_pay_times = self.df(pay_times)
        level = np.sum(df_pay_times * year_fractions)
        swap_fwd = (df_start_time - df_pay_times[-1]) / level
        gamma = np.sum(df_pay_times * year_fractions * self.beta(expiry, pay_times)) / level

        # Results
        num = self.df(pay_date) * (gamma - self.beta(expiry, pay_date))
        den = df_pay_times[-1] * self.beta(expiry, pay_times[-1]) + level * swap_fwd * gamma
        a = num / den
        b = self.df(pay_date) / level - a * swap_fwd

        return {"a" : a, "b" : b}

## Replication Method : The Carr-Madan Formula

From the previous parts, we have :  

\begin{equation}
\begin{split}
PV & = LVL(0,T_0, T_1) \times E^{Q^{LVL}} \left( \frac{f\left(S(T_f, T_0, T_1)\right) \times B(T_f, T_p) }{LVL(T_f,T_0, T_1)} \right) \\
& = LVL(0,T_0, T_1) \times E^{Q^{LVL}} \left[ f(S(T_f, T_0, T_1)) × g(S(T_f, T_0, T_1) \right] \\
& = LVL(0,T_0, T_1) \times E^{Q^{LVL}} \left[ h(S(T_f, T_0, T_1))  \right] \\
\end{split}
\end{equation}

<br>Where : $g$ is the TSR mean reversion linear function and $h=f×g$

We use the Carr-Madan formula to compute the expected value part as follows:

$$
h(S(T_f)) = h(K^*) + h'(K^*) × \left(S(T_f) - K^*\right) + ∫_{-∞}^{K^*}h''(k) \times (k-S(T_f))^+dk + ∫_{K^*}^{+∞}h''(k) \times (S(T_f) - k)^+dk \\
$$

<br>In particular, for CMS forwards :     
* $h(s) = s × (as + b) = as^2 + bs$
* $h'(s) = 2as + b$
* $h''(s) = 2a$
* $K^* = S(0, T_0, T_1)$

Therefore :
\begin{equation}
\begin{split}
CMS_0(T_f, T_0, T_1, T_p) & = \frac{PV}{B(0, T_p)}\\
&= \frac{LVL(0, T_0, T_1)}{B(0, T_p)} \left( \left(aS(0, T_0, T_1)^2+bS(0, T_0, T_1)\right) + ∫_{-∞}^{S(0, T_0, T_1)}2a \times Put_{Swaption}^{OTM}(T_f, k)dk + ∫_{S(0, T_0, T_1)}^{+∞}2a \times Call_{Swaption}^{OTM}(T_f, k)dk \right) \\
\end{split}
\end{equation}

$CMS_t(T_f, T_0, T_1, T_p)$ is always greater than $S(t, T_0, T_1)$, and hence, subject to a positive convexity adjustment

<br>On the other hand, for CMS caplets with strike $K$ :
* $h(s) = (s-K)^+ × (as + b)$
* $h'(s) = (2as + b - aK) × 1_{s > K}$
* $h''(s) = 2a × 1_{s > K}$
* $K^* = K$

Since $h$, and its derivatives, are not continuous and smooth in $K$, the proof shows that we can replace $h'(K^*) × \left(S(T_f) - K^*\right)$ in the Carr-Madan method by its right-sided derivative $h'_+(K^*) × \left(S(T_f) - K^*\right)^+$. This yields to :

$$
PV = LVL(0,T_0, T_1) \times \left( (aK+b) × Call_{Swaption}(T_f, K) + ∫_{K}^{+∞}2a \times Call_{Swaption}^{OTM}(T_f, k)dk \right)
$$

CMS floorlets can then be computed using the Call / Put parity...

In this project, we will consider the normal (Bachelier) model as market model for swaptions :

In [2]:
import numpy as np
from scipy.stats import norm
import math

class swaption :
    """
    Vanilla instrument "Swaption"
    """
    def __init__(self, payer_receiver, expiry, tenor, strike, notional = 1.):
        """
        Init method
        """
        self.payer_receiver = payer_receiver
        self.expiry = expiry
        self.strike = strike
        self.notional = notional
        self.start_time = expiry + 2./365.
        # Fixed leg payment dates : Annually
        self.pay_times = np.arange(self.start_time + 1, expiry + tenor + 1)
        self.year_fractions = np.ones(len(self.pay_times))

    def set_market_data(self, df, normal_vol):
        """
        Store DFs values to save computation time
        """
        self.df_exp_time = df(self.expiry)
        self.df_start_time = df(self.start_time)
        self.df_pay_times = df(self.pay_times)
        self.normal_vol = normal_vol

    def level(self) :
      """
      The level function : Sum of DCF(T(i)) * B(0, T(i)) where T(i) are the fixed leg pay dates
      """
      return np.sum(self.df_pay_times * self.year_fractions)

    def forward(self):
        """
        The forward level of the swap rate
        """
        return (self.df_start_time - self.df_pay_times[-1]) / self.level()

    def pv_underlying(self):
        """
        PV of the underlying swap
        """
        phi = 1. if (self.payer_receiver.upper()=='PAYER') else -1.
        return phi * self.notional * self.level() * (self.forward() - self.strike)

    def market_price(self):
        """
        Swaption price under the Normal model
        """
        phi = 1 if (self.payer_receiver.upper()=='PAYER') else -1
        lvl = self.level()
        fwd = self.forward()
        if (self.expiry==0 or self.normal_vol==0) : return max(phi * (fwd - self.strike), 0)
        sqrt_V2T = self.normal_vol * math.sqrt(self.expiry)
        d = (fwd - self.strike) / sqrt_V2T
        return self.notional * lvl * sqrt_V2T * (phi * d * norm.cdf(phi * d) + norm.pdf(d))

    def market_fwd_price(self):
        """
        Swaption forward price under the Normal model
        """
        phi = 1 if (self.payer_receiver.upper()=='PAYER') else -1
        lvl = self.level()
        fwd = self.forward()
        if (self.expiry==0 or self.normal_vol==0) : return max(phi * (fwd - self.strike), 0)
        sqrt_V2T = self.normal_vol * math.sqrt(self.expiry)
        d = (fwd - self.strike) / sqrt_V2T
        return self.notional * sqrt_V2T * (phi * d * norm.cdf(phi * d) + norm.pdf(d))

Then, we implement the Carr-Madan replication method to price CMS forwards and Caplets / Floorlets :

In [3]:
from scipy.integrate import quad

class replication_method :
    """
    CMS forwards and CMS Caplets / Floorlets prices using the Carr-Madan replication by OTM Swaptions
    """
    def __init__(self, tsr_model) :
        """
        Init method : Takes the TSR model as input
        """
        self.tsr_model = tsr_model

    def replication_price(self, payoff_type, expiry, tenor, pay_date, vol_skew, strike = 0):
        """
        CMS forwards and CMS Caplets / Floorlets prices using the Carr-Madan replication by OTM Swaptions
        payoff_type : Forward, Caplet, or Floorlet
        vol_skew : A function that takes the strike as input, and output the equivalent normal_vol(expiry, strike)
        """
        # df
        df = self.tsr_model.df

        # Dates
        start_time = expiry + 2./365.
        pay_times = np.arange(start_time + 1, expiry + tenor + 1)
        year_fractions = np.ones(len(pay_times))

        # Intermidiate Variables
        df_start_time = df(start_time)
        df_pay_times = df(pay_times)
        df_pay_date = df(pay_date)
        level = np.sum(df_pay_times * year_fractions)
        swap_fwd = (df_start_time - df_pay_times[-1]) / level

        # TSR coefficients
        tsr_coeffs = self.tsr_model.tsr_coeffs(expiry, tenor, pay_date)
        a = tsr_coeffs["a"]
        b = tsr_coeffs["b"]

        # Swaption Prices
        swopt_pay = swaption("Payer", expiry, tenor, swap_fwd)
        swopt_pay.set_market_data(df, vol_skew(swap_fwd))
        swopt_rec = swaption("Receiver", expiry, tenor, swap_fwd)
        swopt_rec.set_market_data(df, vol_skew(swap_fwd))
        def swopt_fwd_price(payer_receiver, k) :
            if payer_receiver.upper() == "PAYER" :
                swopt_pay.strike = k
                swopt_pay.normal_vol = vol_skew(k)
                return swopt_pay.market_fwd_price()
            else :
                swopt_rec.strike = k
                swopt_rec.normal_vol = vol_skew(k)
                return swopt_rec.market_fwd_price()

        # PayOff CMS Fwd functions
        payoff = lambda s : a * s**2 + b * s
        payoff_d = lambda s : 2 * a * s + b
        payoff_sd = lambda s : 2 * a

        # Intermidiate Functions : OTM Swaptions called in the integration
        otm_put_part = lambda k : payoff_sd(k) * swopt_fwd_price("Receiver", k)
        otm_call_part = lambda k : payoff_sd(k) * swopt_fwd_price("Payer", k)

        # CMS Fwd price by Replication : We use -100% and 100% as boundaries of the SciPy quad integration method
        rep_price = payoff(swap_fwd) + quad(otm_put_part, -1., swap_fwd)[0] + quad(otm_call_part, swap_fwd, 1.)[0]
        cms_fwd = level * rep_price / df_pay_date

        if payoff_type.upper()=="FORWARD" :
            return {"Swap Fwd" : round(100 * swap_fwd, 4), "CMS Fwd" : round(100 * cms_fwd, 4), "Conv. Adj." : round(100 * (cms_fwd - swap_fwd), 4)}
        
        elif payoff_type.upper()=="CAPLET" or payoff_type.upper()=="FLOORLET" :
            # PayOff CMS Caplet functions
            payoff = lambda s : (a * s + b) * (s - strike) if s >= strike else 0
            payoff_d = lambda s : (2 * a * s + b - a * strike if s >= strike else 0)
            payoff_sd = lambda s : (2 * a if s >= strike else 0)

            # CMS Caplet price by Replication
            rep_price = (a * strike + b) * swopt_fwd_price("Payer", strike) + quad(otm_call_part, strike, 1)[0]

            if payoff_type.upper()=="CAPLET" :
                return round(level * rep_price, 4)
            else :
                # Call/Put Parity : CMS Floorlet price by Replication
                return round(level * rep_price - df_pay_date * (cms_fwd - strike), 4)
        
        else :
            print("Undefined PayOff type. Possible values : Forward, Caplet, or Floorlet.")
            return None

## Zero-Coupon Rates Curve

We define a Zero-Coupon curve class to construct discount factor curves :

In [4]:
import numpy as np
from scipy.interpolate import interp1d

class zc_curve :
    """
    Zero-Coupon Rates Curve
    """
    def __init__(self, maturities, zc_rates):
        """
        Init method : Takes the IR market tenors and the bootstrapped ZC rates
        """
        self.maturities = maturities
        self.zc_rates = zc_rates
        # Cubic Interpolation & Extrapolation
        self.zc_rates_interp = interp1d(maturities, zc_rates, kind='cubic', fill_value="extrapolate")

    def df(self, T):
        """
        Zero-Coupon Discount Factors
        """
        return np.exp(-self.zc_rates_interp(T)*T)

## Volatility Skew

We define a Volatility Skew class to construct normal vol skews through interpolation / extrapolation methods :

In [5]:
class vol_skew :
    """
    Normal Vol Skew
    """
    def __init__(self, strikes, vol_data) :
        """
        Init method : Takes the market market strikes and their implied normal volatilities
        """
        self.strikes = strikes
        self.vol_data = vol_data

    def normal_vols_skew(self, strike) :
        """
        A function that takes the strike as input, and output the equivalent normal_vol(expiry, strike)
        Cubic interpolation & Linear extrapolation
        """
        if strike < self.strikes[0] or strike > self.strikes[-1] :
            return interp1d(self.strikes, self.vol_data, kind='linear', fill_value="extrapolate")(strike)
        return interp1d(self.strikes, self.vol_data, kind='cubic', fill_value="extrapolate")(strike)

## Application

Let us compute EUR 10Y CMS forwards and EUR 10Y CMS Caplets / Floorlets fixed in 5Y and paid in 6Y, asof 1st Febrary 2024 :

In [6]:
# IR yield curve asof 1st Febrary 2024 (EURIBOR 6M)
maturities = np.array([0.5, 1, 2, 5, 6, 8, 10, 15, 20, 30])
zc_rates = np.array([3.84, 3.41, 2.84, 2.48, 2.47, 2.49, 2.52, 2.6, 2.53, 2.28]) / 100
yc = zc_curve(maturities, zc_rates)

# Normal Market Volatilities asof 1st Febrary 2024 (EURIBOR 6M) : Expiry 5Y x Tenor 10Y
expiry = 5.
tenor = 10.
pay_date = 6.
strikes_5y10y = np.array([1.18, 1.68, 2.18, 2.68, 3.68, 4.68, 5.18]) / 100.
normal_vols_5y10y = np.array([84.70, 83.81, 83.76, 84.74, 89.82, 98.07, 102.91]) / 10000.
vol_skew_5y10y = lambda k : vol_skew(strikes_5y10y, normal_vols_5y10y).normal_vols_skew(k)

# TSR Model : Swap 5Yx10Y to be paid in 6Y
tsr = tsr_model(yc.df)
tsr_coeffs = tsr.tsr_coeffs(expiry, tenor, pay_date)

# CMS Convexity Adjustment : Swap 5Yx10Y to be paid in 6Y
rep_method = replication_method(tsr)
replication_result = rep_method.replication_price("Forward", expiry, tenor, pay_date, vol_skew_5y10y)
print("5Yx10Y CMS Convexity Ajustment (%) :", replication_result)

5Yx10Y CMS Convexity Ajustment (%) : {'Swap Fwd': 2.6873, 'CMS Fwd': 2.8742, 'Conv. Adj.': 0.1869}


In [7]:
# ATM 5Yx10Y CMS Caplet & Floorlet with a payment date of 6Y
print("The ATM 5Yx10Y CMS Caplet & Floorlet prices :")
print("----------------------------------------------------")
print("Caplet price (Bps) : ", 10000. * rep_method.replication_price("Caplet", expiry, tenor, pay_date, vol_skew_5y10y, replication_result["CMS Fwd"] / 100))
print("Floorlet price (Bps) : ", 10000. * rep_method.replication_price("Floorlet", expiry, tenor, pay_date, vol_skew_5y10y, replication_result["CMS Fwd"] / 100))

The ATM 5Yx10Y CMS Caplet & Floorlet prices :
----------------------------------------------------
Caplet price (Bps) :  67.0
Floorlet price (Bps) :  67.0


In [8]:
# 5Yx10Y CMS Caplet & Floorlet with a payment date of 6Y and a strike != ATM
strike = 0.02
print("The 5Yx10Y CMS Caplet & Floorlet prices with a strike of {} :".format(strike))
print("-------------------------------------------------------------")
print("Caplet price (Bps) : ", 10000. * rep_method.replication_price("Caplet", expiry, tenor, pay_date, vol_skew_5y10y, strike))
print("Floorlet price (Bps) : ", 10000. * rep_method.replication_price("Floorlet", expiry, tenor, pay_date, vol_skew_5y10y, strike))

The 5Yx10Y CMS Caplet & Floorlet prices with a strike of 0.02 :
-------------------------------------------------------------
Caplet price (Bps) :  110.0
Floorlet price (Bps) :  34.0
